In [3]:
# import pandas as pd
# import numpy as np
# import os
# import librosa
# import pydub.silence
# import pydub as pb
# from joblib import Parallel, delayed
# echonest = pd.read_csv(os.getcwd() + "/data/echonest.csv",low_memory=False)
# tracks = pd.read_csv(os.getcwd() + "/data/tracks.csv",low_memory=False)

# # <!-- - label using threshold for silence - maybe ~95%
# # - use orignal data for labeling and serpeated data for labeling -->
# labeling_data = os.getcwd() + "/data/data"
# print(os.path.join(labeling_data,'000002.mp3'))
# # try with audio with min silence
# audio1  = pb.AudioSegment.from_mp3(os.path.join(labeling_data,'000005_bass.mp3'))
# silent_ranges = pb.silence.detect_silence(audio1,silence_thresh=-40)        # silence  < thresh dbfs             (0 is max loud)
# total_silence = sum((end - start) for start, end in silent_ranges)
# total_duration = len(audio1)
# print(total_silence / total_duration) # silence percent

# # audio with mostly silence
# audio  = pb.AudioSegment.from_mp3(os.path.join(labeling_data,'000140_drums.mp3'))
# silent_ranges = pb.silence.detect_silence(audio,silence_thresh=-45)        # silence  < thresh dbfs             (0 is max loud)
# total_silence = sum((end - start) for start, end in silent_ranges)
# total_duration = len(audio)
# (print(total_silence / total_duration)) # silence percent

# def process_file(base_path, file, threshold):
#     _aud = pb.AudioSegment.from_mp3(os.path.join(base_path, file))
#     silence = pb.silence.detect_silence(_aud, silence_thresh=-45)
#     silence = sum((e - s) for s, e in silence)
#     percent = silence / len(_aud)
#     _label = 1 if percent <= threshold else 0
#     return [file, _label]

# def lableler(base_path, threshold, num_files=None):
#     files = os.listdir(base_path)[:num_files] if num_files else os.listdir(base_path)
#     results = Parallel(n_jobs=-1)(delayed(process_file)(base_path, file, threshold) for file in files)
#     _labels = pd.DataFrame(results, columns=['filename', 'label'])
#     return _labels
# # labels = lableler(labeling_data,0.8,50)
# labels = lableler(labeling_data, threshold=0.8,num_files=1)
# labels,audio1,audio,echonest,tracks

# labels.to_csv(os.path.join(os.getcwd(),'data\labels.csv'))

instead of training a whole new model to detect these instruments <br>
we can just use demucs to seperate and the above method to detect labels for instruments

then seperate the others if possible
 

In [12]:
# !demucs -d cuda "data/data/000002.mp3" 

seperate other instruments using unet


In [10]:
import torch
import torch.nn as nn
import torchaudio

from torchaudio.transforms import MelSpectrogram
from torchvision.transforms.functional import center_crop
from torch.utils.data import Dataset

import albumentations as A
import torch
from albumentations.pytorch import ToTensorV2

from PIL import Image
# main_source https://github.com/kmaninis/OSVOS-PyTorch/blob/master/dataloaders/helpers.py
import os.path
import time
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torchvision.transforms import CenterCrop, Pad
from torch.utils.data import DataLoader
from tqdm import tqdm

In [9]:
torch.cuda.is_available()

True

In [2]:
class CNNBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=0):
        super(CNNBlock, self).__init__()

        self.seq_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.seq_block(x)
        return x


In [3]:
class CNNBlocks(nn.Module):
    """
    Parameters:
    n_conv (int): creates a block of n_conv convolutions
    in_channels (int): number of in_channels of the first block's convolution
    out_channels (int): number of out_channels of the first block's convolution
    expand (bool) : if True after the first convolution of a blocl the number of channels doubles
    """
    def __init__(self,
                 n_conv,
                 in_channels,
                 out_channels,
                 padding):
        super(CNNBlocks, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(n_conv):

            self.layers.append(CNNBlock(in_channels, out_channels, padding=padding))
            # after each convolution we set (next) in_channel to (previous) out_channels
            in_channels = out_channels

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [4]:
class Encoder(nn.Module):
    """
    Parameters:
    in_channels (int): number of in_channels of the first CNNBlocks
    out_channels (int): number of out_channels of the first CNNBlocks
    padding (int): padding applied in each convolution
    downhill (int): number times a CNNBlocks + MaxPool2D it's applied.
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 padding,
                 downhill=4):
        super(Encoder, self).__init__()
        self.enc_layers = nn.ModuleList()

        for _ in range(downhill):
            self.enc_layers += [
                CNNBlocks(n_conv=2, in_channels=in_channels, out_channels=out_channels, padding=padding),
                nn.MaxPool2d(2, 2)
            ]

            in_channels = out_channels
            out_channels *= 2
        # doubling the dept of the last CNN block
        self.enc_layers.append(CNNBlocks(n_conv=2, in_channels=in_channels,
                                         out_channels=out_channels, padding=padding))

    def forward(self, x):
        route_connection = []
        for layer in self.enc_layers:
            if isinstance(layer, CNNBlocks):
                x = layer(x)
                route_connection.append(x)
            else:
                x = layer(x)
        return x, route_connection


In [5]:
class Decoder(nn.Module):
    """
    Parameters:
    in_channels (int): number of in_channels of the first ConvTranspose2d
    out_channels (int): number of out_channels of the first ConvTranspose2d
    padding (int): padding applied in each convolution
    uphill (int): number times a ConvTranspose2d + CNNBlocks it's applied.
    """
    def __init__(self,
                 in_channels,
                 out_channels,
                 exit_channels,
                 padding,
                 uphill=4):
        super(Decoder, self).__init__()
        self.exit_channels = exit_channels
        self.layers = nn.ModuleList()

        for i in range(uphill):

            self.layers += [
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
                CNNBlocks(n_conv=2, in_channels=in_channels,
                          out_channels=out_channels, padding=padding),
            ]
            in_channels //= 2
            out_channels //= 2

        # cannot be a CNNBlock because it has ReLU incorpored
        # cannot append nn.Sigmoid here because you should be later using
        # BCELoss () which will trigger the amp error "are unsafe to autocast".
        self.layers.append(
            nn.Conv2d(in_channels, exit_channels, kernel_size=1, padding=padding),
        )


    def forward(self, x, routes_connection):
        # pop the last element of the list since
        # it's not used for concatenation
        routes_connection.pop(-1)
        for layer in self.layers:
            if isinstance(layer, CNNBlocks):
                # center_cropping the route tensor to make width and height match
                routes_connection[-1] = center_crop(routes_connection[-1], x.shape[2])
                # concatenating tensors channel-wise
                x = torch.cat([x, routes_connection.pop(-1)], dim=1)
                x = layer(x)
            else:
                x = layer(x)
        return x


In [6]:
class UNET(nn.Module):
    def __init__(self,
                 in_channels,
                 first_out_channels,
                 exit_channels,
                 downhill,
                 padding=0
                 ):
        super(UNET, self).__init__()
        self.encoder = Encoder(in_channels, first_out_channels, padding=padding, downhill=downhill)
        self.decoder = Decoder(first_out_channels*(2**downhill), first_out_channels*(2**(downhill-1)),
                               exit_channels, padding=padding, uphill=downhill)

    def forward(self, x):
        enc_out, routes = self.encoder(x)
        out = self.decoder(enc_out, routes)
        return out


if __name__ == "__main__":   #test

    unet = UNET(1, 64, 1, padding=0, downhill=4)
    test = torch.rand((3, 1, 572, 572))
    print("out unet shape is", unet(test).shape)

out unet shape is torch.Size([3, 1, 388, 388])


Config


In [ ]:
IMAGE_HEIGHT = 388
IMAGE_WIDTH = 388
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ROOT_DIR = os.getcwd() +"/data/others/spectrograms"
EPOCHS = 30
LEARNING_RATE = 1e-4
PAD_MIRRORING = 92
SAVE_MODEL_PATH = os.getcwd() + "/output/models"
SAVE_IMAGES_PATH =os.getcwd() + "/output/segmented_images"
CHECKPOINT = os.getcwd() + "/checkpoints"
CHECKPOINT = None    # remove after chekpointing


train_transform = A.Compose(
    [
        A.Resize(IMAGE_HEIGHT, IMAGE_WIDTH),
        # add  more that do not alter audio signals
        A.Normalize(
            mean=(0, 0, 0),
            std=(1, 1, 1),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transforms = A.Compose(
    [
        A.Resize(IMAGE_HEIGHT, IMAGE_WIDTH),
        A.Normalize(
            mean=(0, 0, 0),
            std=(1, 1, 1),
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)


In [ ]:
class AudDataSet(Dataset):
    def __init__(self, directory, transform=None, precomputed_spectrograms=True):
        self.directory = directory
        self.transform = transform
        self.precomputed_spectrograms = precomputed_spectrograms
        self.files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self, idx):
        file_path = os.path.join(self.directory, self.files[idx])
        
        if self.precomputed_spectrograms:
        # Load the spectrogram
            spectrogram = torch.load(file_path)
        else:
                # Load the raw audio
            waveform, sample_rate = torchaudio.load(file_path)
                
            if self.transform:
                spectrogram = self.transform(waveform)
            else:
                    # Default to MelSpectrogram if no transform is provided
                spectrogram = MelSpectrogram()(waveform)
            
        return spectrogram.squeeze(0)  # Remove the channel dimension

In [ ]:
# converts a torch.Tensor to np.array, removes the batch_dim, if array has more than one channel
# it transposes it from (CHANNELS, HEIGHT, WIDTH) to (HEIGHT, WIDTH, CHANNELS)
def tens2image(im):
    im = im.cpu()
    # removing batch size
    tmp = np.squeeze(im.numpy())
    # if greyscale
    if tmp.ndim == 2:
        return tmp
    else:
        # in order to perform np.subtract and to plot images, we have to transfer channel to the last dimension
        return tmp.transpose((1, 2, 0))

def overlay_mask(im, ma, color=np.array([255, 0, 0]) / 255.0):
    assert np.max(im) <= 255, "RGB channels' value cannot exceed 255"

    # float to bool means: each not-0 is set to 1, each 0 is kept 0
    ma = (ma > 0.5).astype(np.float32)
    im = im.astype(np.uint8)

    alpha = 0.5

    # fg = im*alpha + np.ones(im.shape)*(1-alpha) * np.array([23,23,197])/255.0
    # im (0, 1)
    # color (0, 1)
    # here we are creating a red transparent filter for the whole image that later will be
    # used only in the perimeter of the masks
    fg = im * alpha + np.ones(im.shape) * 255 * (1 - alpha) * color  # np.array([0,0,255])/255.0
    # Whiten background
    alpha = 1
    bg = im.copy()

    # create a simil_fg where fg is set to 0 where the mask is 0
    # substitute the values of the gt_image where mask != 0 with the values of the simil_fg where values are != 0

    # ma == 0 filters all the contour pixels of the mask
    # + np.ones(im[ma == 0].shape) * (1 - alpha) if alpha < 1 is used to whiten the background
    # if alpha=1 (default), + np.ones(im[ma == 0].shape) * (1 - alpha) is dropped

    bg[ma == 0] = im[ma == 0] * alpha + np.ones(im[ma == 0].shape) * 255 * (1 - alpha)

    bg[ma == 1] = fg[ma == 1]

    # cv2.findContours(image, mode, method)
    contours, _ = cv2.findContours(ma.copy().astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # cv2.drawContours(image, contours, colourIdx, color, thickness)
    cv2.drawContours(bg, contours, -1, (0, 0, 0), 1)

    return bg

# rescales the image from (0, 1) to (0, 255)
def inv_normalize(im):
    assert np.max(im) <= 1 and np.min(im) >= 0, "Image is not scaled between 0 and 1"
    # im.min() = im.max() float32 (normalization not done channel-wise)

    return im * 255

def get_loaders(
        db_root_dir,
        batch_size,
        train_transform,
        val_transform,
        num_workers=4,
        pin_memory=True,):
    
    train_ds = AudDataSet(train=True, db_root_dir=db_root_dir, transform=train_transform,
                          pad_mirroring=PAD_MIRRORING)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = AudDataSet(train=False, db_root_dir=db_root_dir, transform=val_transform,
                        pad_mirroring=PAD_MIRRORING)

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        # I have to shuffle otherwise the saved_images are retrieved from
        # the "bike-packing" class that is composed by very similar images
        # otherwise it would be unnecessary
        shuffle=True,
    )

    return train_loader, val_loader

# define train_loop
def train_loop(model, loader, optim, loss_fn, scaler, pos_weight=False):
    loop = tqdm(loader)
    loss_20_batches = 0
    loss_epoch = 0
    for idx, (image, mask) in enumerate(loop):
        # transferring data to cpu or gpu
        image = image.to(DEVICE)
        mask = mask.float().unsqueeze(dim=1).to(DEVICE)

        # float16 training: reduces the load to the VRAM and speeds up the training
        with torch.amp.autocast():
            out = model(image)
            if pos_weight:
                loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=(mask==0.).sum()/mask.sum())
            loss = loss_fn(out, mask)
            loss_20_batches += loss
            loss_epoch += loss

        # backpropagation
        # check docs here https://pytorch.org/docs/stable/amp.html
        optim.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()

        # update tqdm loop
        if idx%20==0:
            loop.set_postfix(loss_20_batches=loss_20_batches.item()/20)
            loss_20_batches = 0

    print(
        f"==> training_loss: {loss_epoch/len(loader):2f}"
    )

def evalution_metrics(model,
                      val_loader,
                      loss_fn,
                      device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    loss_epoch = 0

    model.eval()
    with torch.no_grad():
        for idx, (image, mask) in enumerate(val_loader):
            image = image.to(device)
            mask = mask.float().unsqueeze(dim=1).to(DEVICE)
            pred = model(image)

            loss_epoch += loss_fn(pred, mask)
            mask_pred = torch.sigmoid(pred)
            mask_pred = (mask_pred > 0.5).float()

            num_correct += (mask_pred == mask).sum()
            num_pixels += torch.numel(mask_pred)
            dice_score += (2 * (mask_pred * mask).sum()) / (
                    (mask_pred + mask).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {(num_correct/num_pixels)*100:.2f}"
    )
    print(
        f"==> valuation_loss: {loss_epoch/len(val_loader):2f}"
    )

    print(f"==> dice_score: {dice_score/len(val_loader)}")

    model.train()

def validation_recall(model,
                      val_loader,
                      device=DEVICE):
    model.eval()
    tot_recall = 0
    with torch.no_grad():
        for image, mask in val_loader:
            image = image.to(device)
            mask = mask.to(device).unsqueeze(dim=1)
            out = torch.sigmoid(model(image))
            out = np.array((out>0.5).cpu(), dtype=np.uint8).reshape(1,-1).squeeze()
            mask = np.array(mask.cpu(), dtype=np.uint8).reshape(1,-1).squeeze()
            recall_batch = recall_score(mask, out)
            tot_recall += recall_batch

    model.train()
    print(f'Recall on validation set is: {tot_recall/len(val_loader)}')

def save_checkpoint(state, folder_path, filename="my_checkpoint.pth.tar"):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    print("=> Saving checkpoint...")
    torch.save(state, os.path.join(folder_path, filename))

def load_model_checkpoint(checkpoint, model):
    print("=> Loading model checkpoint...")
    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint["state_dict"])

def load_optim_checkpoint(checkpoint, optim):
    print("=> Loading optimizer checkpoint...")
    checkpoint = torch.load(checkpoint)
    optim.load_state_dict(checkpoint["optimizer"])

def save_images(model, loader, folder, epoch, device, num_images, pad_mirroring):
    print("=> Saving images...")

    path = os.path.join(folder, f"epoch_{epoch + 1}")
    if not os.path.exists(path):
        os.makedirs(path)

    model.eval()

    with torch.no_grad():
        for idx, (images, masks) in enumerate(loader):
            if idx < num_images:
                images = images.to(device)
                outs = torch.sigmoid(model(images))
                outs = (outs > 0.5).float()
                if pad_mirroring:
                    images = CenterCrop((IMAGE_HEIGHT, IMAGE_WIDTH))(images)
                # if the batch_size is > 1 we take just the first image/mask

                # plotting the first image/mask per batch
                image = images[0]
                mask = masks[0]
                out = outs[0]

                image = tens2image(image)
                mask = tens2image(mask)
                out = tens2image(out)

                img_gt = overlay_mask(inv_normalize(image), mask)
                img_out = overlay_mask(inv_normalize(image), out)

                fig = plt.figure(figsize=(10, 7))
                rows = 1
                columns = 2

                fig.add_subplot(rows, columns, 1)
                plt.imshow(img_gt)
                plt.axis('off')
                plt.title("Input image and ground_truth mask")

                fig.add_subplot(rows, columns, 2)
                plt.imshow(img_out)
                plt.axis('off')
                plt.title("Input image and predicted_mask")

                fig.savefig(f'{path}/image_{idx}.png')

                plt.cla()
                plt.close(fig)

            else:
                break

    model.train()

def predict_image(model, image, val_transform, folder, image_title, pad_mirroring):
    path = os.path.join(folder)
    if not os.path.exists(path):
        os.makedirs(path)

    model.eval()

    with torch.no_grad():
        model = model.to(DEVICE)
        image = val_transform(image=image)["image"].to(DEVICE).unsqueeze(dim=0)
        if pad_mirroring:
            image = Pad(padding=pad_mirroring, padding_mode="reflect")(image)
        start = time.time()
        mask = torch.sigmoid(model(image))
        mask = (mask > 0.5).float()
        end = time.time()
        print("Inference time is {:2f}".format(end-start))


        image = tens2image(image)
        mask = tens2image(mask)
        pred = overlay_mask(inv_normalize(image), mask)


        fig = plt.figure(figsize=(10, 7))
        plt.imshow(pred)
        plt.axis('off')
        plt.title("Test Prediction")

        fig.savefig(f"{path}/{image_title}.jpg")

def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from torch.optim import Adam

# we are going to train and compare 3 models:
# 1) input and target 388x388 --> apply mirroring to input to widen it to 572
# 2) input and target 572x572 --> on the mask during forward()
# 2) input and target 572x572, model with padding.



loss_fn = torch.nn.BCEWithLogitsLoss()
scaler = torch.amp.GradScaler()
model = UNET(3, 64, 1, padding=0, downhill=4).to(DEVICE)
optim = Adam(model.parameters(), lr=LEARNING_RATE)

if CHECKPOINT:
    load_model_checkpoint(CHECKPOINT, model)
    load_optim_checkpoint(CHECKPOINT, optim)

train_loader, val_loader = get_loaders(db_root_dir=ROOT_DIR, batch_size=8, train_transform=train_transform,
                                       val_transform=val_transforms, num_workers=4)
for epoch in range(17, EPOCHS):

    print(f"Training epoch {epoch+1}/{EPOCHS}")
    train_loop(model=model, loader=train_loader, loss_fn=loss_fn, optim=optim, scaler=scaler, pos_weight=False)

    print("Computing valuation metrics on val_loader...")

    evalution_metrics(model, val_loader, loss_fn, device=DEVICE)
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optim.state_dict(),
    }

    save_checkpoint(checkpoint, folder_path=SAVE_MODEL_PATH,
                    filename=f"checkpoint_epoch_{epoch+1}.pth.tar")
    save_images(model=model, loader=val_loader, folder=SAVE_IMAGES_PATH,
                epoch=epoch, device=DEVICE, num_images=10, pad_mirroring=PAD_MIRRORING)
    